In [1]:
#init
import torch, numpy, sys
sys.path.append("..")
from chatbot_end2end.model.supervised import Supervised
from nlptools.utils import Config

cfg = Config("../config/cornell.yml")
model = Supervised.build(cfg)

/usr/lib/python3.7/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.25.2) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)
2019-05-16 21:46:28,153 | INFO: Optimizer: adam with learning_rate: 0.0001


In [2]:
model.skill.model.config

{'encoder': {'vocab_size': 30522,
  'hidden_size': 512,
  'intermediate_size': 2048,
  'num_hidden_layers': 3,
  'bidirectional': True},
 'decoder': {'intermediate_size': 2048, 'num_hidden_layers': 3}}

In [6]:
dialog = iter(model.generator).next()
dialog.to("cuda:0")
response = dialog["response_cornell"].data
response_mask = dialog["response_mask_cornell"].data
utterance = dialog["utterance"].data
utterance_mask = dialog["utterance_mask"].data
sentiment = dialog["sentiment"].data
print(utterance, utterance.shape)
print(utterance_mask, utterance_mask.shape)
print(response, response.shape)
print(response_mask, response_mask.shape)
print(sentiment, sentiment.shape)

tensor([[ 101, 2074, 2061,  ..., 1045, 2074,  102],
        [ 101, 2040, 2515,  ...,    0,    0,    0],
        [ 101, 2061, 2017,  ..., 2009, 1012,  102],
        ...,
        [ 101, 1045, 3685,  ..., 2027, 2097,  102],
        [ 101, 2064, 2017,  ...,    0,    0,    0],
        [ 101, 2017, 2113,  ..., 1045, 2342,  102]], device='cuda:0') torch.Size([80, 20])
tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1],
        ...,
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1]], device='cuda:0') torch.Size([80, 20])
tensor([[ 101, 3425, 2525,  ...,  102,    0,    0],
        [ 101, 1045, 2031,  ..., 1012,  102,    0],
        [ 101, 2092, 1010,  ..., 2000, 2147,  102],
        ...,
        [ 101, 2129, 2071,  ...,    0,    0,    0],
        [ 101, 2057, 2024,  ..., 1010, 2057,  102],
        [ 101, 2198, 1012,  ...,    0,    0,    0]], device='cuda:0') torch.Size([80, 20])
tensor([[1, 1, 1

In [11]:
import nlptools.zoo.encoders.transformer
import nlptools.zoo.encoders.lstm
import importlib
importlib.reload(nlptools.zoo.encoders.transformer)
importlib.reload(nlptools.zoo.encoders.lstm)
from nlptools.zoo.encoders.transformer import TransformerEncoder
from nlptools.zoo.encoders.lstm import LSTMEncoder

model.skill.model.encoder = LSTMEncoder(**model.skill.model.config["encoder"]).to("cuda:0")
encoder_out = model.skill.model.dialog_embedding(utterance, utterance_mask, sentiment)
prev_output = response.data[:, :-1]
target_output = response.data[:, 1:]
print(encoder_out.shape)
print(prev_output.shape)
print(target_output.shape)

tensor([[ 101, 2074, 2061,  ..., 1045, 2074,  102],
        [ 101, 2040, 2515,  ...,    0,    0,    0],
        [ 101, 2061, 2017,  ..., 2009, 1012,  102],
        ...,
        [ 101, 1045, 3685,  ..., 2027, 2097,  102],
        [ 101, 2064, 2017,  ...,    0,    0,    0],
        [ 101, 2017, 2113,  ..., 1045, 2342,  102]], device='cuda:0') torch.Size([80, 20])
tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1],
        ...,
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1]], device='cuda:0') torch.Size([80, 20])
tensor([[[-0.4099,  0.3876, -0.2835,  ..., -1.3094, -2.5112,  0.6827],
         [-0.4982,  0.9905,  1.3793,  ...,  0.0310,  1.4094, -0.8233],
         [ 0.1872, -0.0559, -1.0342,  ...,  0.8720, -0.9012, -0.2197],
         ...,
         [-0.0315, -0.2239, -1.6710,  ..., -1.7988, -0.9550, -0.1954],
         [-0.4982,  0.9905,  1.3793,  ...,  0.0310,  1.4094, -0.8233],
         [ 

TypeError: cannot unpack non-iterable NoneType object

In [ ]:
for name, parameter in model.skill.model.named_parameters():
    print(name, parameter.requires_grad)

In [ ]:
for parameter in model.skill.model.decoder.fc3.parameters():
    print(parameter.requires_grad)

In [ ]:
target_output = target_output.unsqueeze(-1).contiguous().view(-1)
target_output.shape

In [ ]:
import nlptools.zoo.encoders.transformer
import nlptools.zoo.encoders.lstm
import importlib
importlib.reload(nlptools.zoo.encoders.transformer)
importlib.reload(nlptools.zoo.encoders.lstm)
from nlptools.zoo.encoders.transformer import TransformerDecoder, TransformerEncoder
from nlptools.zoo.encoders.lstm import LSTMDecoder, LSTMEncoder

embedding = model.skill.model.encoder.embedding
decoder_hidden_layers = 1
decoder_attention_heads = 3
decoder_hidden_size = 3072
dropout = 0
decoder = TransformerDecoder(embedding, decoder_hidden_layers, decoder_attention_heads, decoder_hidden_size, dropout)
decoder = decoder.to("cuda:0")
output = decoder(prev_output, encoder_out, utterance_mask)
output.shape

In [ ]:
decoder